This notebook is meant for combining and reinterpolating spectra to prepare them for further analysis, as well as gerenating a three-column text or CSV file for reporting the final data. Any adjustments to boost cohesion or overall ease of use are welcome and appreciated!

-Armaan Goyal

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
import scipy
from scipy.interpolate import interp1d
import pandas as pd

plt.style.use(astropy_mpl_style)

def data_from_text(filename):
    data = np.genfromtxt(filename, usecols = (0, 1))
    wav, flux = np.split(data, 2, axis = 1)
    flux = np.reshape(flux, len(flux))
    wav = np.reshape(wav, len(wav))
    return wav, flux

The spec_interp routine is intended to reinterpolate and/or trim spectra to prepare them for combination with other files (either another exposure for the same data or another color channel). 

It is important to use the the same step size to reinterpolate all spectra that you intend to combine so that the final output file will have a uniform grid and will be consistent across all channels and exposures (a step size around 0.5-0.8 is recommended but anything close to 1 should provide quality resolution for the data). 

The interp parameter represents the type of interpolation desired and may be set to any of the functions described in the scipy.interpolate.interp1d documentation at https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html ("cubic" is highly reccommended).

The low and high parameters are used to trim spectra to a desired wavelength region. If combining spectra from different color channels, choose their boundaries such that there is no overlap between them (max value for blue channel should be equal to min value of the red). 

In [4]:
def spec_interp(filename, interp, step, **kwargs):
    '''
    Parameters:
    -----------
    filename(str): name of input text file
    interp (str): type of interpolation desired, options at https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html
    step (float): step size for interpolation, value of 1 or less is reccommended
    low/high (float, optional): wavelength bounds for trimming of spectrum
    save_as (str, optional): desired filename for output
    
    Returns:
    --------
    Reinterpolated/trimmed spectrum (with optional output as text file).
    
    '''
    low = kwargs.get("low", None)
    high = kwargs.get("high", None)
    save_as = kwargs.get("save_as", None)
    wav, flux = data_from_text(filename)
    if low:
        bool_arr = (wav >= low)
        wav, flux = wav[bool_arr], flux[bool_arr]
    if high:
        bool_arr = (wav <= high)
        wav, flux = wav[bool_arr], flux[bool_arr]
    interpolant = interp1d(wav, flux, kind = interp)
    new_wav = np.arange(min(wav), max(wav), step)
    new_flux = interpolant(new_wav)
    if save_as:
        output = np.column_stack((new_wav, new_flux))
        np.savetxt(save_as, output)
    return (new_wav, new_flux)

spec_interp("finalspec_red_1", interp = "cubic", .85, low = 5600, high = 10000, save_as = "final_spec_red_1_interp")

The spec_combine function allows for average/median combining of multiple spectra with an optional sigma clipping algorithm. Ensure that all spectra that are being passed into the routine have the same bounds and the same number of points. 

In [5]:
def spec_combine(filenames, combine, **kwargs):
    '''    
    Parameters:
    -----------
    filenames (list of str): names of input text files
    combine (str): type of combination desired, "avg" for average and "med" for median
    sigclip (tuple of float, optional): low/high sigma values for sigma clipping algorithm (absolute value, do not input negative number for lower sigma)
    save_as (str, optional): desired filename for output
    
    Returns:
    --------
    Combined spectrum (with optional output as text file).
    
    '''
    sigclip = kwargs.get("sigclip", None)
    save_as = kwargs.get("save_as", None)
    arr = []
    all_inds = []
    for file in filenames:
        wav, flux = data_from_text(file)
        if sigclip:
            med, std = np.mean(flux), np.std(flux)
            clip = (flux >= med - sigclip[0]*std) & (flux <= med + sigclip[1]*std)
            inds = np.where(clip)
            all_inds.append(inds)
        else:
            arr.append(flux)
    if sigclip:
        all_inds = np.hstack(all_inds)
        all_inds = np.unique(all_inds)
        for file in filenames:
            wav, flux = data_from_text(file)
            wav, flux = wav[all_inds], flux[all_inds]
            arr.append(flux)
    if combine == "med":
        comb_flux = np.median(np.vstack(arr), axis = 0)
    if combine == "avg":
        comb_flux = np.mean(np.vstack(arr), axis = 0)
    if save_as:
        output = np.column_stack((wav, comb_flux))
        np.savetxt(save_as, output)
    return (wav, comb_flux)

spec_combine(["finalspec_red_1", "finalspec_red_2"], combine = "avg", sigclip = (3, 3), save_as = "red_combined")

The followign is a simple routine for concatenating text files corresponding to spectra for different color channels. Ensure again before using the function that the wavelength values of the channels do not overlap.

In [6]:
def spec_concat(filenames, save_as):
    wav_arr = []
    flux_arr = []
    for file in filenames:
        wav, flux = data_from_text(file)
        wav_arr.append(wav)
        flux_arr.append(flux)
    wav_arr = np.hstack(wav_arr)
    flux_arr = np.hstack(flux_arr)
    output = np.column_stack((wav_arr, flux_arr))
    np.savetxt(save_as, output)
    return output

spec_concat(["blue_spec", "red_spec"], "finalspec")

Here are tasks for consolidating the final object and error spectra into a three-column text file and (if desired) saving the data in CSV form as well.

In [7]:
def three_column(data, error, save_name):
    wav, flux = data_from_text(data)
    wav, err = data_from_text(error)
    output = np.column_stack((wav, flux, err))
    np.savetxt(save_name, output)
    
three_column("finalspec_ext_norm", "error_ext_norm_norm", "grb141031b")

def make_csv(file, save_as):
    data = np.genfromtxt(file)
    pd.DataFrame(data).to_csv(save_as)
    return data

make_csv("grb141031b", "grb141031b.csv")

Finally, this combine_errs taks should only be used if provided with individual error spectra for multiple exposures without access to the original data. This routine will combine the error spectra in accordance with proper error propagation and provides an option to save the output as a new file. This error spectrum should then be trimmed/reinterpolated accordingly so that it aligns with the object spectrum.

In [8]:
def combine_errs(files, **kwargs):
    save_as = kwargs.get("save_as", None)
    arr = []
    for f in files:
        wav, flux = data_from_text(f)
        arr.append(flux)
    num = len(arr)
    arr = np.sqrt(np.sum(np.vstack(arr)**2, axis = 0))/num
    if save_as:
        output = np.column_stack((wav, arr))
        np.savetxt(save_as, output)
    return arr

combine_errs(["error_1", "error_2"], "final_error")